# McWhorter Problem
<cite>[McWhorter and Sunada][1]</cite> propose an analytical solution to the two-phase flow equation. A one-dimensional problem was considered which describes the flow of two incompressible, immiscible fluids through a porous medium, where the wetting phase (water) displaces the non-wetting fluid (air or oil) in the horizontal direction (without the influence of gravity).

![mcWhorter_concept_schematic.png](figures/mcWhorter_concept.png)

### Material Parameters

| Property  | Symbol | Value | Unit |
|----------:|:------:|-------|------|
| Porosity | $\phi$ | $$0.15$$ | 1 |
| Intrinsic permeability | $K$ | $$1.0\cdot 10^{-10}$$ | $m^2$|
| Residual saturation of the wetting phase | $$s_\mathrm{L}^{res}$$ | $$0.02$$ | 1 |
| Residual saturation of the non-wetting phase | $$s_\mathrm{G}^{res}$$ | $$0.001$$ | 1 |
| Dynamic viscosity of the wetting phase	|$\mu_\mathrm{L}$|$$1.0\cdot 10^{-3}$$|Pa s|
| Dynamic viscosity of the non-wetting pha  |$\mu_\mathrm{G}$  |$$5.0\cdot 10^{-3}$$|Pa s|
| Brooks and Corey model parameter: entry pressure | $p_b$ | $$5000$$| Pa |
| Brooks and Corey model parameter: pore size distribution index |$\lambda$ | $$3.0$$ | 1 |

### Problem Parameters

| Property  | Symbol | Value | Unit |
|----------:|:------:|-------|------|
| Initial saturation | $$s_\mathrm{L}(t=0)$$ | 0.05 | 1 |
| Injection boundary saturation | $$s_\mathrm{L}(x=0)$$ | 0.8 | 1 |


[1]: https://doi.org/10.1029/WR026i003p00399

In [1]:
param = {"mun": 1.e-3,
         "muw": 1.e-3,
         "K": 1e-10,
         "phi": 0.3,
         "lambda_": 2,
         "Pe": 5000,
         "S0": 0.9,
         "Si": 0.2}

range = {"mun": (0.1, 1.e4),
         "muw": (0.1, 1.e4),
         "K": (1e-12, 1e-8),
         "phi": (0.05, 0.3),
         "lambda_": (1.5, 3),
         "Pe": (1e3, 1e5),
         "S0": (0.51, 1),
         "Si": (0, 0.5)}

scale = {"mun": "log",
         "muw": "log",
         "K": "log",
         "phi": "linear",
         "lambda_": "linear",
         "Pe": "log",
         "S0": "linear",
         "Si": "linear"}

## Analytical solution
The analytical solution is calculated according to <cite>[McWhorter and Sunada][1]</cite>.

[1]: https://doi.org/10.1029/WR026i003p00399

In [2]:
from mcworther import BrooksCorey, McWorther

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def plot_f(mun, muw, K, phi, lambda_, Pe, S0, Si):
    model = BrooksCorey(Pe, lambda_)
    problem = McWorther(model, phi, K, muw, mun, S0, Si)
    problem.plot_solution()

# Create Sliders for interactive plot
sliders = {}
for key in param.keys():
    if scale[key] == "log":
        sliders[key] = widgets.FloatLogSlider(
            value=param[key], base=10, min=np.log10(range[key][0]),
            max=np.log10(range[key][1]), step=0.01, description=key
        )
    else:
        sliders[key] = widgets.FloatSlider(
            value=param[key], min=range[key][0], max=range[key][1],
            step=0.01, description=key
        )

# Show plot and sliders
ui = widgets.VBox(list(sliders.values()))
out = widgets.interactive_output(plot_f, sliders)
#layout = widgets.HBox([out, ui])
#display(layout)

display(out)
display(ui)

Output()

## Numerical solutions

For the numerical solution, we compare the Thermal-2-Phase-Hydro-Mechanical (TH2M) and the Two-phase Flow formulation.

The first step is to create a matching mesh that capture the penetration depth of the wetting fluid. 

In [ ]:
import pyvista as pv
from helper import mesh1d

#parameters defined by the current slider settings
params = {k: v.value for k, v in sliders.items()}

#calculate analytical solution with the 
model = BrooksCorey(params["Pe"], params["lambda_"])
problem = McWorther(model, params["phi"], params["K"], params["muw"], params["mun"], params["S0"], params["Si"])
x_ref, Sw_ref = problem.get_solution()

depth = max(x_ref)      # penetration depth of the wetting fluid
factor = 5.0   # mesh should be factor times larger than the depth 
Nel = 100         # number of mesh elements

print(f"penetration depth of the wetting fluid: {depth}")

mesh = mesh1d(point_a=(0.0, 0.0, 0.0), point_b=(depth*factor, 0.0, 0.0), num_points=Nel+1)
mesh = pv.UnstructuredGrid(mesh)
mesh.save(r"mesh.vtu")

coords = {'x1': depth*factor, 'x2': 0.5*depth*factor}
prj_from_template(coords, ["mcwt.template"], ["mcwt.gml"])

penetration depth of the wetting fluid: 0.08369312905953712


Because the OGS models use the capillary pressure as a primary variable, the boundary and initial conditions need to be converted from saturations to pressures.

In [4]:
param["pci"] = model.pc(param["Si"])
param["pc0"] = model.pc(param["S0"])

In [5]:
import ogstools as ogs
from template import prj_from_template

# Simulate problem with TwoPhaseFlowPP and TH2M

# template_files = [r"TwoPhase_mcwt_line.template", r"mcWorther_h2.template"]
# prj_files = [r"TwoPhase_mcwt_line_test.prj", r"mcWorther_h2_test.prj"]

template_files = [r"TwoPhase_mcwt_line.template"]
prj_files = [r"TwoPhase_mcwt_line_test.prj"]


for template_file, prj_file in zip(template_files, prj_files):
    prj_from_template(param, template_file, prj_file)
    prj = ogs.Project(input_file=prj_file, output_file=prj_file)
    prj.run_model()

ModuleNotFoundError: No module named 'ogstools'

## Evaluation and Results

In [ ]:
import numpy as np
import pyvista as pv
from helper import plot_with_error


# 1. Plot analytical solution
plotter = plot_with_error(x_ref, Sw_ref, "C0-",
                          "Analytical solution",
                          "Distance $x$ (m)",
                          "Wetting saturation $S_w$ (-)",
                          "Error (Numerical - Analytical) (-)")

# 2. Read results from TwoPhaseFlowPP

#labels = ["TwoPhaseFlowPP", "TH2M"]
#results = ["twophaseflow_test_t_1000.000000.vtu", "mcWhorter_h2_newton_ts_110_t_1000.000000.vtu"]

labels = ["TwoPhaseFlowPP"]
results = ["twophaseflow_test_t_1000.000000.vtu"]


i = 1
for label, result in zip(labels, results):

    mesh = pv.read(result)
    print(mesh.point_data.keys())

    Sw=mesh["saturation"]
    x=mesh.points[:,0]

    ind = np.argsort(x)
    x = x[ind]
    Sw = Sw[ind]

    plotter.append(x, Sw, f"C{i}x", label)
    i = i + 1 

# show plot

plotter.plot([0, depth*1.2])

## Literature

McWhorter, D. B., and D. K. Sunada (1990), Exact integral solutions for two-phase flow, Water Resour. Res., 26(3), 399–413, <cite>[doi:10.1029/WR026i003p00399][1]</cite>. 

[1]: https://doi.org/10.1029/WR026i003p00399
